## NYCbuswatcher
#### Step 2. Read all the parquets and repartition by route-hour using Pyspark

In [1]:
import os, json
import pandas as pd

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, DoubleType, TimestampType, LongType

spark = SparkSession.builder \
  .master("local") \
  .appName("nyc_reparition_individual_parquets_by_route_hour_1") \
  .config("spark.executor.cores", 3) \
  .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/15 12:56:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)

In [3]:
in_path="/Volumes/nice/bigdata/bus_depot/bus-data-migrations/nyc/data/out/nyc_shipments_as_parquets/"

df = spark.read.parquet(in_path)
print((df.count(), len(df.columns)))

(2442497, 23)


In [4]:
df.printSchema()

root
 |-- route: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- route_long: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- service_date: string (nullable = true)
 |-- trip_id: string (nullable = true)
 |-- gtfs_shape_id: string (nullable = true)
 |-- route_short: string (nullable = true)
 |-- agency: string (nullable = true)
 |-- origin_id: string (nullable = true)
 |-- destination_name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- bearing: double (nullable = true)
 |-- progress_rate: string (nullable = true)
 |-- progress_status: string (nullable = true)
 |-- vehicle_id: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- next_stop_id: string (nullable = true)
 |-- next_stop_eta: string (nullable = true)
 |-- next_stop_d_along_route: double (nullable = true)
 |-- next_stop_d: double (nullable = true)
 |-- gtfs_block_id: string (nullable = true)



In [5]:
# https://sparkbyexamples.com/spark/pyspark-to_timestamp-convert-string-to-timestamp-type/
from pyspark.sql.functions import *

#Timestamp String to DateType
new_df = df.withColumn("timestamp",to_timestamp("timestamp"))
new_df.printSchema()

root
 |-- route: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- route_long: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- service_date: string (nullable = true)
 |-- trip_id: string (nullable = true)
 |-- gtfs_shape_id: string (nullable = true)
 |-- route_short: string (nullable = true)
 |-- agency: string (nullable = true)
 |-- origin_id: string (nullable = true)
 |-- destination_name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- bearing: double (nullable = true)
 |-- progress_rate: string (nullable = true)
 |-- progress_status: string (nullable = true)
 |-- vehicle_id: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- next_stop_id: string (nullable = true)
 |-- next_stop_eta: string (nullable = true)
 |-- next_stop_d_along_route: double (nullable = true)
 |-- next_stop_d: double (nullable = true)
 |-- gtfs_block_id: string (nullable = true)



In [6]:
out_path="/Volumes/nice/bigdata/bus_depot/bus-data-migrations/nyc/data/out/nyc_shipments_as_parquets_partitioned/"
os.makedirs(out_path, exist_ok=False)

In [7]:
# https://stackoverflow.com/questions/52527888/spark-partition-data-writing-by-timestamp/52528333#52528333
    
new_df \
    .withColumn("year", year(col("timestamp"))) \
    .withColumn("month", month(col("timestamp"))) \
    .withColumn("day", dayofmonth(col("timestamp"))) \
    .withColumn("hour", hour(col("timestamp"))) \
    .repartition("year", "month", "day", "hour") \
    .write \
    .mode('overwrite')\
    .partitionBy("year", "month", "day", "hour","route_short") \
    .parquet(out_path)

22/04/15 12:56:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
